In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM

In [4]:
device = f"cuda:0"
#from bits4llama.llama import load_quant
#model = load_quant("pretrain_weights/vicuna/vicuna-7b-v1.1", "pretrain_weights/vicuna/vicuna-7b-v1.1/vicuna7b-4bit-128g.pt", 4, 128,device=device).to(device)
# model = LlamaForCausalLM.from_pretrained(
#     "internlm/internlm-7b", low_cpu_mem_usage=True, torch_dtype=torch.float16).cuda()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 device = f"cuda:0"                                                                           │
│   2 #from bits4llama.llama import load_quant                                                     │
│ ❱ 3 model = load_quant("pretrain_weights/vicuna/vicuna-7b-v1.1", "pretrain_weights/vicuna/vi     │
│   4 # model = LlamaForCausalLM.from_pretrained(                                                  │
│   5 #     "internlm/internlm-7b", low_cpu_mem_usage=True, torch_dtype=torch.float16).cuda()      │
│   6                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'load_quant' is not defined

In [4]:
from fastchat.model.model_adapter import (
    load_model,
    get_conversation_template,
)
model_path = 'pretrain_weights/vicuna-7b-v1.1'
model, tokenizer = load_model(
    model_path,
    "cuda",
    1,
    load_8bit=False
)

pretrain_weights/vicuna-7b-v1.1
pretrain_weights/vicuna-7b-v1.1/pytorch_model.bin
False
pretrain_weights/vicuna-7b-v1.1/pytorch_model.bin.index.json
True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
conv = get_conv_template('vicuna_v1.1').copy()
conv.append_message(conv.roles[0], """Given the passage: \"\"\"This article presents a model of general-purpose computing on a semantic network substrate. The concepts presented are applicable to any semantic network representation. However, due to the standards and technological infrastructure devoted to the Semantic Web effort, this article is presented from this point of view. In the proposed model of computing, the application programming interface, the run-time program, and the state of the computing virtual machine are all represented in the Resource Description Framework (RDF). The implementation of the concepts presented provides a practical computing paradigm that leverages the highly-distributed and standardized representational-layer of the Semantic Web.\"\"\" Please check whether the passage is relative to the question: what is Semantic Network . Return Yes or No.""")
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
input_ids = tokenizer([prompt]).input_ids
input_ids = torch.as_tensor(input_ids).cuda()
#output_ids = model.generate(input_ids,max_length=600)
# print(prompt)
# print(output_ids[0][len(input_ids[0]):])
# print(tokenizer.decode(output_ids[0][len(input_ids[0]):]))
# print("==================")
output_ids = model.generate(input_ids,return_dict_in_generate=True,output_scores=True,max_length=600)
print(prompt)
print(output_ids.sequences[0][len(input_ids[0]):])
print(tokenizer.decode(output_ids.sequences[0][len(input_ids[0]):]))
print("==================")

A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Given the passage: """This article presents a model of general-purpose computing on a semantic network substrate. The concepts presented are applicable to any semantic network representation. However, due to the standards and technological infrastructure devoted to the Semantic Web effort, this article is presented from this point of view. In the proposed model of computing, the application programming interface, the run-time program, and the state of the computing virtual machine are all represented in the Resource Description Framework (RDF). The implementation of the concepts presented provides a practical computing paradigm that leverages the highly-distributed and standardized representational-layer of the Semantic Web.""" Please check whether the passage is relative to the question: what is Semantic Network . Return Yes o

In [25]:
3869

3869

In [19]:
torch.argmax(output_ids.scores)

tensor(3869, device='cuda:0')

In [23]:
output_ids.scores[0][:,[1939,3869,29889]].softmax(1)

tensor([[0.4036, 0.5962, 0.0000]], device='cuda:0', dtype=torch.float16)

In [88]:
import numpy as np

In [93]:
data = np.array(df)

In [97]:
new_data = []
for (_id,paper_id,abstract,key_word_1,key_word_2,key_word_3,key_word_4,
    key_word_5,key_word_6,key_word_7,key_word_8,key_word_9,key_word_10) in data:
    new_data.append([paper_id,abstract.replace('\n',' '),key_word_1,key_word_2,key_word_3,key_word_4,
    key_word_5,key_word_6,key_word_7,key_word_8,key_word_9,key_word_10])

In [99]:
column_titles = ['paper_id', 'abstract', 'key_word_1', 'key_word_2', 'key_word_3',
                 'key_word_4', 'key_word_5', 'key_word_6', 'key_word_7', 'key_word_8',
                 'key_word_9', 'key_word_10']
df = pd.DataFrame(new_data, columns=column_titles)

In [101]:
df.to_csv('data/unArXiv.key_word_from_abstract.csv')

In [69]:
import torch.nn.functional as F

In [70]:
lm_logits      = output_ids.scores[0][:,[1939,3869]] # the first output and the logists for No/Yes
log_softmax    = F.log_softmax(lm_logits, dim=-1)

In [75]:
log_softmax.norm(dim=1,keepdim=True)

tensor([[1.1025]], device='cuda:0', dtype=torch.float16)

In [5]:
def vicuna_check_is_relative_template(title_ids, context_ids,MAX_SEQUENCE_LEN=512):
        title_ids = title_ids[1:]    # remove the <s>
        context_ids = context_ids[1:]  # remove the <s>
        '''template is 
        <s>A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Here is a passage about [<title_ids>]:\n"""\n<context_ids>\n"""\n Based on the given passage, formulate a question that aligns most accurately with the primary fact disclosed within the text. ASSISTANT:
        '''
        base = [1, 319, 13563, 1546, 263, 12758, 1404, 322, 385, 23116, 21082, 20255, 29889, 450, 20255, 4076, 8444, 29892, 13173,
                        29892, 322, 1248, 568, 6089, 304, 278, 1404, 29915, 29879, 5155, 29889, 3148, 1001, 29901]
        a = base + [11221, 278, 13382, 6160, 15945] 
        b = [15945, 29908, 3529, 1423, 3692, 278, 13382, 338, 6198, 304, 278, 1139, 29901, 825, 338]
        c = [869, 7106, 3869, 470, 1939,29889,  319, 1799, 9047, 13566, 29901]
        
        complete_sequence  = a + context_ids + b + title_ids + c
        exceed_length = len(complete_sequence) - MAX_SEQUENCE_LEN
        if exceed_length >0: context_ids = context_ids[:-exceed_length]
        return a + context_ids + b + title_ids + c

In [17]:
tokenizer(["A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Here is a passage from a scientific paper:"]).input_ids

[[1,
  319,
  13563,
  1546,
  263,
  12758,
  1404,
  322,
  385,
  23116,
  21082,
  20255,
  29889,
  450,
  20255,
  4076,
  8444,
  29892,
  13173,
  29892,
  322,
  1248,
  568,
  6089,
  304,
  278,
  1404,
  29915,
  29879,
  5155,
  29889,
  3148,
  1001,
  29901,
  2266,
  338,
  263,
  13382,
  515,
  263,
  16021,
  5650,
  29901]]

In [6]:
print('We demonstrate that assuming the "discrete" vacuum geometry in the\nMinkowskian Higgs model with vacuum BPS monopole solutions can justify the\nDirac fundamental quantization of that model. The important constituent of this\nquantization is getting various rotary effects, including collective solid\nrotations inside the physical BPS monopole vacuum, and just assuming the\n"discrete" vacuum geometry seems to be that thing able to justify these rotary\neffects. More precisely, assuming the "discrete" geometry for the appropriate\nvacuum manifold implies the presence of thread topological defects (side by\nside with point hedgehog topological defects and walls between different\ntopological domains) inside this manifold in the shape of specific\n(rectilinear) threads: gauge and Higgs fields located in the spatial region\nintimately near the axis $z$ of the chosen (rest) reference frame. This serves\nas the source of collective solid rotations proceeding inside the BPS monopole\nvacuum suffered the Dirac fundamental quantization. It will be argued that\nindeed the first-order phase transition occurs in the Minkowskian Higgs model\nwith vacuum BPS monopoles quantized by Dirac. This comes to the coexistence of\ntwo thermodynamic phases inside the appropriate BPS monopole vacuum. There are\nthe thermodynamic phases of collective solid rotations and superfluid potential\nmotions.'.replace('\n',' '))

We demonstrate that assuming the "discrete" vacuum geometry in the Minkowskian Higgs model with vacuum BPS monopole solutions can justify the Dirac fundamental quantization of that model. The important constituent of this quantization is getting various rotary effects, including collective solid rotations inside the physical BPS monopole vacuum, and just assuming the "discrete" vacuum geometry seems to be that thing able to justify these rotary effects. More precisely, assuming the "discrete" geometry for the appropriate vacuum manifold implies the presence of thread topological defects (side by side with point hedgehog topological defects and walls between different topological domains) inside this manifold in the shape of specific (rectilinear) threads: gauge and Higgs fields located in the spatial region intimately near the axis $z$ of the chosen (rest) reference frame. This serves as the source of collective solid rotations proceeding inside the BPS monopole vacuum suffered the Dir

In [38]:


tokenizer.decode(vicuna_only_passage_template(tokenizer.encode("what is mother"), tokenizer.encode("mother is one of partents")))

'<s>A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions. USER: Here is a passage from a scientific paper:""" mother is one of partents""" Based on the given passage, formulate a question that aligns most accurately with the primary fact disclosed within the text. ASSISTANT:'